In [1]:
# import libraries
import random
import json
import pickle
import numpy as np
import tensorflow as tf

# importing NLTK library for natural language processing
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# Initializing WordNetLemmatizer for word lemmatization
lemmatizer = WordNetLemmatizer()

# Loading intents JSON file containing patterns for different intents
intents = json.loads(open('/content/drive/MyDrive/Chatbot/intents.json').read())


# Initialize lists and variables for storing words, classes, and documents
words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']


# Preprocessing the intents and extracting words, classes, and documents
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


# Lemmatizing words and removing ignore letters
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))
classes = sorted(set(classes))

# Saving preprocessed words and classes to files using pickle
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Initializing training data
training = []
outputEmpty = [0] * len(classes)

# Processing documents to create training data
for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

# Shuffling training data and converting it into numpy array
random.shuffle(training)
training = np.array(training)

# Splitting training data into features (X) and labels (Y)
trainX = training[:, :len(words)]
trainY = training[:, len(words):]

# Building the neural network model using TensorFlow Keras
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

# Compiling the model with SGD optimizer and categorical crossentropy loss function
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Training the model with the training data
hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)

# Saving the trained model
model.save('chatbot_model.h5', hist)
print('Done')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Epoch 1/200
4/4 [==============================] - 4s 7ms/step - loss: 1.4198 - accuracy: 0.3333
Epoch 2/200
4/4 [==============================] - 0s 4ms/step - loss: 1.3506 - accuracy: 0.3333
Epoch 3/200
4/4 [==============================] - 0s 4ms/step - loss: 1.3005 - accuracy: 0.5556
Epoch 4/200
4/4 [==============================] - 0s 4ms/step - loss: 1.2257 - accuracy: 0.4444
Epoch 5/200
4/4 [==============================] - 0s 4ms/step - loss: 1.2591 - accuracy: 0.3889
Epoch 6/200
4/4 [==============================] - 0s 4ms/step - loss: 1.1992 - accuracy: 0.5556
Epoch 7/200
4/4 [==============================] - 0s 4ms/step - loss: 1.1817 - accuracy: 0.5000
Epoch 8/200
4/4 [==============================] - 0s 4ms/step - loss: 1.1557 - accuracy: 0.5000
Epoch 9/200
4/4 [==============================] - 0s 5ms/step - loss: 1.0644 - accuracy: 0.6111
Epoch 10/200
4/4 [==============================] - 0s 4ms/step - loss: 0.9907 - accuracy: 0.6667
Epoch 11/200
4/4 [===========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# import libraries
import random
import json
import pickle
import numpy as np
import nltk

from nltk.stem import WordNetLemmatizer
from keras.models import load_model

# Initialize WordNetLemmatizer for word lemmatization
lemmatizer = WordNetLemmatizer()

# Load intents JSON file containing predefined intents for the chatbot
intents = json.loads(open('/content/drive/MyDrive/Chatbot/intents.json').read())

# Load preprocessed words and classes from pickle files
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Load the trained chatbot model
model = load_model('chatbot_model.h5')

# Function to clean up a sentence by tokenizing and lemmatizing it
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

# Function to create a bag of words representation for a sentence
def bag_of_words (sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

# Function to predict the class/intent of a sentence
def predict_class (sentence):
    bow = bag_of_words (sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes [r[0]], 'probability': str(r[1])})
    return return_list

# Function to get a response based on the predicted intent
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice (i['responses'])
            break
    return result

print("GO! Bot is running!")
#print("Press 0 if you dont't want to chat with our ChatBot")

# Loop to continuously interact with the chatbot
while True:
    message = input("You:")
    ints = predict_class (message)
    res = get_response (ints, intents)
    print (res)


GO! Bot is running!
